In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os, sys
import pandas as pd

In [2]:
sys.path.append(os.path.abspath('../'))

import sfr_methods


In [ ]:
plt.style.use('seaborn')
plt.style.use('seaborn-colorblind')

In [ ]:
usable_paths = np.load('../usable_v3_paths.npy')

In [ ]:
def integrated_sfr(galdf, galdict, avg, threshold):
    df = galdf.copy()
    
    df = df.dropna()
    df = df[(df['$r/r_e$'] < 1.5) & (df['$r/r_e$'] > 0.1)]
    df = df[(df['Comp'] == 0) & (df['AGN'] == 0) & (df['Seyfert'] == 0) & (df['Liner'] == 0)]
    df = df[(df['Spiral Arm ' + str(threshold)] == True) | (df['Nonspiral Arm ' + str(threshold)] == True)]
    
    spiral_df = df[df['Spiral Arm ' + str(threshold)] == True]
    nspiral_df = df[df['Nonspiral Arm ' + str(threshold)] == True]
    
    spiral_sfr, sig_spiral_sfr = sfr_methods.get_sfr(spiral_df.index, galdf, galdict, avg=avg)
    nspiral_sfr, sig_nspiral_sfr = sfr_methods.get_sfr(nspiral_df.index, galdf, galdict, avg=avg)
    combined_sfr, sig_combined_sfr = sfr_methods.get_sfr(df.index, galdf, galdict, avg=avg)
    
    return spiral_sfr, sig_spiral_sfr, nspiral_sfr, sig_nspiral_sfr, combined_sfr, sig_combined_sfr

In [ ]:
def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [ ]:
def calculate_stdv(series):
    mean = series.mean()
    
    summ = 0
    for element in series:
        summ = summ + (element - mean)**2
    
    return np.sqrt(summ / len(series))

In [ ]:
dict_list = []
df_list = []

for path in usable_paths:
    filename = path.split('/')[-1].split('.')[0]
    
    galdict = load_dict('../Form_Resources/Dicts_Avg_SFR_Thresholds//' + filename + '.dict.pkl')
    galdf = pd.read_pickle('../Form_Resources/DFs/' + filename + '.df.pkl')
    
    dict_list.append(galdict)
    df_list.append(galdf)

In [ ]:
main_df = pd.DataFrame(dict_list)

In [ ]:
main_df

In [ ]:
thresholds = []

for i in range(1, 8):
    for j in range(1, 8):
        thresholds.append((i, j))

In [ ]:
for threshold in thresholds:
    main_df['(spiral - nspiral)/nspiral ' + str(threshold)] = (main_df['avg_spiral_sfr ' + str(threshold)] - main_df['avg_nonspiral_sfr ' + str(threshold)]) / main_df['avg_nonspiral_sfr ' + str(threshold)]
    main_df['sig(spiral - nspiral)/nspiral ' + str(threshold)] = main_df['(spiral - nspiral)/nspiral ' + str(threshold)] * ((main_df['sig_avg_spiral_sfr ' + str(threshold)] / main_df['avg_spiral_sfr ' + str(threshold)])**2 + (main_df['sig_avg_nonspiral_sfr ' + str(threshold)] / main_df['avg_nonspiral_sfr ' + str(threshold)])**2)**0.5

In [ ]:
main_df

In [ ]:
thresholds_odd = []

for i in range(1, 8, 2):
    for j in range(1, 8, 2):
        thresholds_odd.append((i, j))

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(42, 32))

for i in range(4):
    for j in range(4):
        filtered_df = main_df[main_df['(spiral - nspiral)/nspiral ' + str(thresholds_odd[i*4 + j])] < 10]
        series = filtered_df['(spiral - nspiral)/nspiral ' + str(thresholds_odd[i*4 + j])]
        axes[i, j].hist(series, bins=20)
        axes[i, j].tick_params(axis="x", labelsize=20)
        axes[i, j].set_yticks([])
        
        mean_series = series.mean()
        median_series = series.median()
        stdv_series = calculate_stdv(series)
        
        axes[i, j].axvline(median_series, linestyle='--', linewidth=3, alpha=0.3, color='k',
                           label='Median = {:.4}%'.format(median_series))
        axes[i, j].axvline(mean_series, linestyle='-', linewidth=2.5, alpha=0.8, color='white',
                           label='Mean = {:.4}%'.format(mean_series))
        axes[i, j].axvspan(mean_series - stdv_series, mean_series + stdv_series, alpha=0.15, color='purple',
                           label='$\sigma$ = {:.4}%'.format(stdv_series))
        
        axes[i, j].set_title('Sample: {} Galaxies'.format(len(series)), fontsize=20)
        axes[i, j].legend(fontsize=20)

        
for i in range(4):
    axes[3, i].set_xlabel('Non-spiral Threshold: ' + str(i * 2 + 1), size=40, labelpad=40)
    
for i in range(4):
    axes[i, 0].set_ylabel('Spiral Threshold: ' + str(i * 2 + 1), size=40, labelpad=50)
    
plt.subplots_adjust(wspace=0.01, hspace=0.2)